In [78]:
from pyspark import SparkContext
import numpy as np
sc = SparkContext.getOrCreate()


In [79]:
def splitAndTake(line):
    arr = line.split('\t')
    try:
        date = str(arr[1])
        #print(date)
        rate = float(arr[2].strip().replace(",","."))
        return (date,rate)
    except:
        return None

In [80]:
def shiftIndex(line):
    return(line[0] + 1, line[1])

In [81]:
def swapIndex(line):
    return(line[1], line[0])

In [82]:
dataRDD = sc.textFile('datasets/DollarDataset.txt')
dataRDD = dataRDD.filter(lambda x: x[1] != "")
dataRDD = dataRDD.map(lambda line: splitAndTake(line))
dataRDD = dataRDD.zipWithIndex()
dataRDD = dataRDD.map(lambda line: swapIndex(line))
dataRDD_copy = dataRDD.map(lambda line: shiftIndex(line))

In [83]:
tmp = dataRDD_copy.join(dataRDD) # join aynı indexlileri birleştiriyor

In [84]:
def findRateDifference(line):
    prev_date = line[1][0][0]
    next_date = line[1][1][0]
    prev_rate = line[1][0][1]
    next_rate = line[1][1][1]
    ratio = next_rate / prev_rate
    return(prev_date, next_date, ratio)

In [85]:
rateRDD = tmp.sortBy(lambda x: x[0])
rateRDD = rateRDD.filter(lambda x: (x[1][0] != None) and (x[1][1] != None))
rateRDD = rateRDD.map(lambda line: findRateDifference(line))
rateRDD = rateRDD.sortBy(lambda x : x[2],ascending=False)
rateRDD.take(10)

[('19-08-1960', '22-08-1960', 3.2142857142857144),
 ('24-01-1980', '25-01-1980', 2.0),
 ('07-08-1970', '10-08-1970', 1.65),
 ('11-06-1979', '12-06-1979', 1.320754716981132),
 ('28-02-1978', '01-03-1978', 1.2987012987012987),
 ('20-09-1977', '21-09-1977', 1.1),
 ('22-10-2008', '23-10-2008', 1.0723684210526314),
 ('28-02-1977', '01-03-1977', 1.0606060606060606),
 ('09-04-1979', '10-04-1979', 1.06),
 ('06-06-1980', '09-06-1980', 1.0583446404341927)]

increase is calculated by dividing next day value to previous day value